<a href="https://colab.research.google.com/github/udituen/lexical_complexity_pred/blob/main/LCP_cmput600.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install gdown
!pip install syllables
!pip -q install transformers
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

**Download Dataset**

In [ ]:
%%bash
rm -r sample_data

mkdir train
cd train
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv?dl=1' -O 'lcp_single_train.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_multi_train.tsv?dl=1' -O 'lcp_multi_train.tsv'
cd ..

mkdir trial
cd trial
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_single_trial.tsv' -O 'lcp_single_trial.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/trial/lcp_multi_trial.tsv' -O 'lcp_multi_trial.tsv'
cd ..

mkdir test
cd test
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test/lcp_single_test.tsv?dl=1' -O 'lcp_single_test.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test/lcp_multi_test.tsv?dl=1' -O 'lcp_multi_test.tsv'
cd ..

mkdir test-labels
cd test-labels
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv?dl=1' -O 'lcp_single_test.tsv'
wget -q 'https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_multi_test.tsv?dl=1' -O 'lcp_multi_test.tsv'
cd ..

mkdir score
mkdir glove-embeddings
mkdir saves

**Import necessaty packages**

In [ ]:
import time
import csv
import wget
import os
import copy
import sqlite3
import syllables
import pytz
from collections import defaultdict
import glob
import sys

import numpy as np
import pandas as pd
import spacy
import torch
from torch import nn
import sklearn
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm, trange
from transformers import BertTokenizer
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader
from torch.utils.data import ConcatDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

Get senses from target word


In [ ]:

import string
from itertools import chain
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic as wnic
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet_ic')

from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, word_tokenize
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.


In [ ]:
semcor_ic  = wnic.ic('ic-semcor.dat')

In [ ]:
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, word_tokenize
SS_PARAMETERS_TYPE_MAP = {'definition':str, 'lemma_names':list,
                          'examples':list,  'hypernyms':list,
                         'hyponyms': list, 'member_holonyms':list,
                         'part_holonyms':list, 'substance_holonyms':list,
                         'member_meronyms':list, 'substance_meronyms': list,
                         'part_meronyms':list, 'similar_tos':list}


In [ ]:
porter = PorterStemmer()
wnl = WordNetLemmatizer()

In [ ]:

def lemmatize(ambiguous_word, pos=None, neverstem=False,
              lemmatizer=wnl, stemmer=porter):
    """
    Tries to convert a surface word into lemma, and if lemmatize word is not in
    wordnet then try and convert surface word into its stem.
    This is to handle the case where users input a surface word as an ambiguous
    word and the surface word is a not a lemma.
    """

    if pos:
        lemma = lemmatizer.lemmatize(ambiguous_word, pos=pos)
    else:
        lemma = lemmatizer.lemmatize(ambiguous_word)
    stem = stemmer.stem(ambiguous_word)
    # Ensure that ambiguous word is a lemma.
    if not wn.synsets(lemma):
        if neverstem:
            return ambiguous_word
        if not wn.synsets(stem):
            return ambiguous_word
        else:
            return stem
    else:
     return lemma

In [ ]:
def synset_properties(synset, parameter):
    """
    Making from NLTK's WordNet Synset's properties to function.
    Note: This is for compatibility with NLTK 2.x
    """
    return_type = SS_PARAMETERS_TYPE_MAP[parameter]
    func = 'synset.' + parameter
    return eval(func) if isinstance(eval(func), return_type) else eval(func)()


In [ ]:
def synset_signature(Targetword, pos=None):

    synsets_signatures = {}
    for synset in wn.synsets(Targetword):
        # If POS is specified.
        if pos and str(synset.pos()) != pos:
            continue
        signature = []
        # Includes definition.
        definition = synset_properties(synset, 'definition')
        signature+=definition
        # Includes examples
        examples = synset_properties(synset, 'examples')
        signature+=list(chain(*[i.split() for i in examples]))
        # Includes lemma_names.
        lemma_names = synset_properties(synset, 'lemma_names')
        signature+= lemma_names
        # Includes lemma_names of hypernyms and hyponyms.
        hyponyms = synset_properties(synset, 'hyponyms')
        hypernyms = synset_properties(synset, 'hypernyms')
        hypohypernyms = hypernyms+hyponyms
        signature+= list(chain(*[i.lemma_names() for i in hypohypernyms]))
        # Removes stopwords.
        signature = [i for i in signature if i not in stopwords]

        # Lemmatized context is preferred over stemmed context.
        signature = [lemmatize(i) for i in signature]
        # Matching exact words may cause sparsity, so optional matching for stems.
        signature = [porter.stem(i) for i in signature]
        synsets_signatures[synset] = signature
    # print(synsets_signatures)

    return synsets_signatures

In [ ]:
def penn2morphy(penntag, returnNone=False):
    morphy_tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return None if returnNone else ''

In [ ]:
def compare_overlaps_simple_lesk(sentence, synsets_sign):
   
    overlaplen_synsets = [] # a tuple of (len(overlap), synset).
    for ss in synsets_sign:
        overlaps = set(synsets_sign[ss]).intersection(sentence)
        overlaplen_synsets.append(ss)

    # Rank synsets from highest to lowest overlap.
    ranked_synsets = sorted(overlaplen_synsets, reverse=True)
  
    return ranked_synsets[0] if len(ranked_synsets) >= 1 else '0'

In [ ]:
def lemmatize_sentence(sentence, neverstem=False, keepWordPOS=False,
                       tokenizer=word_tokenize, postagger=pos_tag,
                       lemmatizer=wnl, stemmer=porter):
  
    words, lemmas, poss = [], [], []
    for word, pos in postagger(tokenizer(sentence)):
        pos = penn2morphy(pos)
        lemmas.append(lemmatize(word.lower(), pos, neverstem,
                                lemmatizer, stemmer))
        poss.append(pos)
        words.append(word)
    if keepWordPOS:
        return words, lemmas, [None if i == '' else i for i in poss]
    return lemmas


In [ ]:
"""
Simple Lesk uses the synset signature (definitions.examples and lemmas) and compare it with the surrounding text.
And like the Original Lesk it returns the highest overlap between the synset signature and the sentence
"""
def simple_lesk(sentence,Targetword,pos=None,sentence_is_lemmatized=False):
    # Ensure that ambiguous word is a lemma.
    Targetword = lemmatize(Targetword)
    # Get the signatures for each synset.
    ss_sign = synset_signature(Targetword, pos)
    # Disambiguate the sense in context.
    if sentence_is_lemmatized:
        sentence = sentence.split()
    else:
        sentence = lemmatize_sentence(sentence)

    # print(sentence)
    highest_overlap = compare_overlaps_simple_lesk(sentence, ss_sign)
    # print(highest_overlap)
    return highest_overlap


In [ ]:
# read data
train_data_single =  pd.read_csv('train/lcp_single_train.tsv',  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
train_data_multi =  pd.read_csv('train/lcp_multi_train.tsv',  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
test_data_single = pd.read_csv('test-labels/lcp_single_test.tsv',  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
test_data_multi = pd.read_csv('test-labels/lcp_multi_test.tsv',  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
# train_pseudo_single= pd.read_csv('pseudo_train.tsv',  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

train_data = train_data_single
train_data = train_data.append(train_data_multi, ignore_index=True)
# pseudo_train = pd.read_csv('pseudo_train.tsv',  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')


In [ ]:
def remove_dataset_errors(data):
    for i, token in enumerate(list(data.token)):
        if isinstance(token, float):
            data.token.iloc[i]="null"
            # print(i,token)
    return data

train_data = remove_dataset_errors(train_data)
# train_pseudo_data = remove_dataset_errors(pseudo_train)
test_data_single = remove_dataset_errors(test_data_single)
test_data_multi = remove_dataset_errors(test_data_multi)




Disambiguate words

In [ ]:
def getSensesFunc(self, h_sentence, token):

  sent = list(h_sentence)
  token = list(token)

  returned_sense = []
  for word, sentence in zip(token, h_sentence):
    # print(sent)
    returned_sense.append(simple_lesk(sentence, word))

  return returned_sense

In [ ]:
# Variables
random_seed = 42
random_state = 42
batch_size = 32
emb_dim = 300
epochs = 5

# Directory to save trained models
model_save_dir = '/content/saves/'

# True if you want to use already processed glove embeddings stored in drive
# False to generate glove embeddings using 'glove.840B.300d.zip' file (Time Consuming)
use_pre_processed_glove_emvedding = True

# Below are preprocessed glove embessing urls | Enter below urls if "use_pre_processed_glove_emvedding = True" to use preprocessed word embeddings
preprocessed_glove_embedding_url_test_single = 'https://drive.google.com/uc?id=1tr6wO5JuOJKo0pVsNdjjcR_4xXwIiimk'
preprocessed_glove_embedding_url_test_multi = 'https://drive.google.com/uc?id=1BbPkLw6Obq8g7jdhMDyn-fcougUcjjE-'
preprocessed_glove_embedding_url_train_multi = 'https://drive.google.com/uc?id=1WeJmYV0yIWPvLp2diaMFvPP1-zfP5nZb'

# Enter below urls/path if "use_pre_processed_glove_emvedding = False" i.e. extract glove embeddings from original corpus (Time Consuming)
glove_embedding_url = 'http://nlp.stanford.edu/data/glove.840B.300d.zip' 
glove_embedding_download_path = '/content/glove-embeddings/glove.840B.300d.txt' 
glove_embedding_save_path = '/content/glove-embeddings/preprocessed_glove_embeddings.csv'#do not change

**Variables used ahead can be changed here**

In [ ]:
# Initialization
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)
print("Device Name: ",torch.cuda.get_device_name())

**Load train and test data from csv files**
> Combine both training datasets of single and multi word tokens






pandas reads "null" word tokens as null values, converting them back to "null" string

In [ ]:
from google.colab import files
base_synset = files.upload()

Saving base_synset_correct.txt to base_synset_correct.txt


**Dataset Class is used to do all the preprocessings required on datasets**

> Commented part of the code was used to generate features that were used during experimentation phase but removed from the final code



In [ ]:
class Dataset():
    def __init__(self, data, split, batch_size=32, max_len=128):
        self.split = split                                          
        self.data = data                           
        self.max_len = max_len                                              self.batch_size = batch_size                                
        
        self.tokens = list(data.token)
        self.sentences = list(data.sentence)
        self.complexity = torch.Tensor(list(data.complexity))
        
        # Glove word embedding for tokens
        self.token_embedding = torch.FloatTensor(self.word_glove_embeddings(data.token).values)

        # Bert Sentence to vector id conversion with attention mask
        self.tokenised_sent_id, self.sent_attention_mask = self.BertSentTokenizer()

        self.getWSD = self.getSenses(data.sentence, data.token)

        # Hand-Crafted Features
        self.senses = self.wordnet_senses(data.token)
        self.token_len = self.token_sizes(data.token)                                                       
        self.syllable_count = self.syllableCount(data.token)                                               
        self.vowel_count = self.vowelCount(data.token)                                                      
        self.token_type = self.get_token_type()                                                            
        self.min_depth = self.getMinDepth(data.sentence, data.token)                                                            
        self.get_max_similarity = self.getMaxSimilarity(data.sentence, data.token)                                                             
        self.get_wup_similarity = self.getWUPSimilarity(data.sentence, data.token)                                                            
        self.get_lin_similarity = self.getLINSimilarity(data.sentence, data.token)                                                            
        self.get_lch_similarity = self.getLChSimilarity(data.sentence, data.token)                                                             


        self.handcrafted_features = torch.FloatTensor(                              # Handcrafted features combined together
                                        np.hstack((
                                            self.min_depth,
                                            self.get_max_similarity,
                                            self.get_wup_similarity,
                                            self.get_lin_similarity,
                                            self.get_lch_similarity,
                                            
                                            # self.token_len,
                                            # self.syllable_count,
                                            # self.vowel_count,
                                            # self.token_type,
                                            # self.senses,                              
                                            ))
                                    )

    def __str__(self):
      return self.get_max_similarity

    def getSenses(self, sentence, token):
      return [i for i in getSensesFunc(self, sentence, token)]

    def wordnet_senses(self, token):
      senses = []
      for word in token:
          if ' ' in word:
              s = 0
              # look if a pair is present
              s += len(wn.synsets(word.lower().replace(' ','_')))
              # look for each word separately
              s += sum([len(wn.synsets(w)) for w in word.lower().split(' ')])
              senses.append(s)
          else:
              syns = wn.synsets(word.lower())
              senses.append(len(syns))
              
          result = np.array(senses)
              
          num_senses = np.expand_dims(result,axis=1)

      return num_senses


    def getMinDepth(self, sentence, token):
      senses = []

      target = getSensesFunc(self, sentence, token)
      min_depth = []

      for syn in target:
        syn = str(syn)
        try:
          if syn == '0':
            min_depth.append(0)
          else:
            t = (f"wn.{syn.lower()}.min_depth()")
            e = eval(t)
            min_depth.append(e)
        except:
          min_depth.append(0)

      mini_depth = np.expand_dims(min_depth, axis=1)
      return np.array(mini_depth)
    

    def getMaxSimilarity(self,sentence, token):

        target = getSensesFunc(self, sentence, token)
        base_concept = base_synset['base_synset_correct.txt'].decode("utf-8").split(", ")

        base = list(base_concept)

        # initialise place holders
        total_scores = []
        result = []
        result_=[]

        # initialise counters
        scores = []

        # track the number of target synsets
        for syn in target:
          try:
            # preprocess target synset, skip those without senses
            syn = str(syn)
            if syn == '0':
              result.append([0.0])
            else:
              t = (f"wn.{syn.lower()}")
              e = eval(t)
              # track the number of base synsets
              for i, c in enumerate(base):
                if c != '':
                  c = (f'wn.synset("{c}")')
                  core = eval(c)
                  if (wn.path_similarity(e, core)) == None:
                    result.append([0.0])
                  else:
                    result.append([wn.path_similarity(e, core)])
          except:
            result.append([0.0])
              
          maximum_sense = max(result)
          result_.append(maximum_sense)
          
          return np.array(result_)
 

    def getWUPSimilarity(self,sentence, token):

        target = getSensesFunc(self, sentence, token)
        base_concept = base_synset['base_synset_correct.txt'].decode("utf-8").split(", ")

        base = list(base_concept)

        # initialise place holders
        total_scores = []
        result = []
        # initialise counters
        wup_sim = []
        # track the number of target synsets
        for syn in target:
          try:
            syn = str(syn)
            if syn == '0':
              result.append([0.0])
            else:
              t = (f"wn.{syn.lower()}")
              e = eval(t)
            # track the number of base synsets
              for i, c in enumerate(base):
                if c != '':
                  c = (f'wn.synset("{c}")')
                  core = eval(c)
                  if (wn.wup_similarity(e, core)) == None:
                    result.append([0.0])
                  else:
                    result.append([wn.wup_similarity(e, core)])
                    # print(f'token: {t} and core: {c} = {wn.wup_similarity(e, core)}')
          except:
            result.append([0.0])

          max_sense = max(result)
          wup_sim.append(max_sense)
        return np.array(wup_sim)


    def getLINSimilarity(self,sentence, token):

        target = getSensesFunc(self, sentence, token)
        base_concept = base_synset['base_synset_correct.txt'].decode("utf-8").split(", ")

        # target = list(target)
        base = list(base_concept)

        # initialise place holders
        total_scores = []
        result = []
        lin_sim=[]
        # initialise counters
        scores = []

        # track the number of target synsets
        for i in target:
          try:
            i = str(i)
            # get the part of speach for target synset
            target_ = i.split(".")
            target_ = target_[1]

            # try:
            if i == '0':
              result.append([0.0])
            else:
              t = (f"wn.{i.lower()}")
              e = eval(t)

              # track the number of base synsets
              for c in base:
                if c != '':
                # get part of speech of core synset, return only exact matches with target synset
                  core_ = c.split(".")
                  core_ = core_[1]

                  if target_ == core_:
                  
                    c = (f'wn.synset("{c}")')
                    core = eval(c)
        
                    if (e.lin_similarity(core, semcor_ic)) == None:
                      result.append([0.0])
                    else:
                      result.append([e.lin_similarity(core, semcor_ic)])
                  else:
                    result.append([0.0])
          except:
            result.append([0.0])

          max_sense = max(result)
          lin_sim.append(max_sense) 
        return np.array(lin_sim)

    # Leacock-Chodorow Similarity
    # based on the shortest path that connects the senses and the maximum depth of the taxonomy in which the senses occur. same pos
    def getLChSimilarity(self,sentence, token):

        # target = target_word['target_synsets_new.txt'].decode("utf-8").split(", ")
        
        target = getSensesFunc(self, sentence, token)
        base_concept = base_synset['base_synset_correct.txt'].decode("utf-8").split(", ")

        # target = list(target)
        base = list(base_concept)

        # initialise place holders
        total_scores = []
        result = []
        lch_sim=[]
        # initialise counters
        scores = []
        # track the number of target synsets
        for syn in target:
          syn = str(syn)
          try:
            if syn == '0':
              result.append([0.0])
            else:
              t = (f"wn.{syn.lower()}")
              e = eval(t)
              # track the number of base synsets
              for i, c in enumerate(base):
                if c != '':
                  c = wn.synset(f"{c}")
                  if (wn.path_similarity(t, c)) == None:
                    result.append([0.0])
                  else:
                    result.append([wn.lch_similarity(t, c)])
          except:
            result.append([0.0])

          max_sense =  max(result)
          lch_sim.append(max_sense)


        return np.array(lch_sim)

    def get_token_type(self):
        token_type = []
        for token in self.tokens:
            if len(token.split(' '))>1:
                token_type.append([0])
            else:
                token_type.append([1])
        print(token_type)
        return np.array(token_type)

    def vowelCount(self, tokens):   
        vowel_count = []
        vowels = ['a','e','i','o','u']

        for token in tokens:
            count = len([1 for c in token if c in vowels])
            vowel_count.append(count)
        
        max_vowel_count=max(vowel_count)
        vowel_count = [[v/max_vowel_count] for v in vowel_count]
        return np.array(vowel_count)

    def syllableCount(self, tokens):
        syllable_count = []
        
        for token in tokens:
            syllable_count.append(syllables.estimate(token))
        
        max_syllable_count = max(syllable_count)
        syllable_count = [[sc/max_syllable_count] for sc in syllable_count]
        
        return np.array(syllable_count)

    def token_sizes(self, tokens):
        max_token_len = 0
        # token length, word frequency, 
        for token in tokens:
            max_token_len = max(max_token_len, len(token))

        tokenSizes = []
        for token in tokens:
            tokenSizes.append( [len(token)/max_token_len] )

        return np.array(tokenSizes)

    def BertSentTokenizer(self):
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        tokenised_sent_id = []
        sent_attention_mask =[]

        for sent, token in zip(self.sentences, self.tokens):
            inp = str(token).lower() + ' [SEP] ' +  str(sent).lower() 
            out = tokenizer.encode_plus(inp, add_special_tokens=True, max_length=self.max_len, padding='max_length', 
                                            return_attention_mask = True, return_tensors = 'pt', truncation = True)
            tokenised_sent_id.append(out['input_ids'])
            sent_attention_mask.append(out['attention_mask'])

        tokenised_sent_id = torch.cat(tokenised_sent_id, dim=0)
        sent_attention_mask = torch.cat(sent_attention_mask, dim=0)

        return tokenised_sent_id, sent_attention_mask

    def generate_word_glove_embeddings(self,tokens):
        print("Filtering token Embeddings ...")
        embedding_dict={}
        with open(glove_embedding_download_path, 'r', encoding="utf-8") as f:
            for line in f:
                try:
                    cols = line.split()
                    token = cols[0]
                    vector = np.asarray(cols[1:],"float32")
                    embedding_dict[token] = vector
                except:
                    flag=1
        
        glove_embeddings=[]
        for multi_word in tokens:
            if len(multi_word.split(' '))==1:
                if token in embedding_dict.keys():
                    glove_embeddings.append(embedding_dict[token])
                else:
                    glove_embeddings.append(np.asarray([0]*300))
                 
            else:
                words = multi_word.split(' ')
                if words[0] in embedding_dict.keys():
                    e1=embedding_dict[words[0]]
                else:
                    e1=np.asarray([0]*300)
                
                if words[1] in embedding_dict.keys():
                    e2=embedding_dict[words[1]]
                else:
                    e2=np.asarray([0]*300)
                
                glove_embeddings.append((e1+e2)/2)

        return pd.DataFrame(np.array(glove_embeddings))

    def word_glove_embeddings(self,tokens):
        if use_pre_processed_glove_emvedding == True:

            if self.split=='single test':
                out='glove-embeddings/preprocessed_glove_embeddings_test_single.csv'
                if not os.path.isfile(out):
                    cmd = "gdown '{url}' -O '{out}'".format(url=preprocessed_glove_embedding_url_test_single, out=out)
                    !eval {cmd}

            elif self.split=='multi test':
                out='glove-embeddings/preprocessed_glove_embeddings_test_multi.csv'
                if not os.path.isfile(out):
                    cmd = "gdown '{url}' -O '{out}'".format(url=preprocessed_glove_embedding_url_test_multi, out=out)
                    !eval {cmd}

            elif self.split=='train':
                out='glove-embeddings/preprocessed_glove_embeddings_train_multi.csv'
                if not os.path.isfile(out):
                    cmd = "gdown '{url}' -O '{out}'".format(url=preprocessed_glove_embedding_url_train_multi, out=out)
                    !eval {cmd}
        else:     
            if not os.path.isfile('/content/glove-embeddings/'+glove_embedding_url.split('/')[-1]):
                print("Downloading glove dataset ... (This may take few minutes)")
                wget.download(url=glove_embedding_url, out='/content/glove-embeddings/'+glove_embedding_url.split('/')[-1])
                print("Download Complete")
                cmd = "unzip '{input}' -d '{out}'".format(input='/content/glove-embeddings/'+glove_embedding_url.split('/')[-1], out='/content/glove-embeddings/')
                !eval {cmd}

            glove_embedding = self.generate_word_glove_embeddings(tokens)
            if self.split=='single test':
                glove_embedding.to_csv(glove_embedding_save_path[:-4]+"_test_single.csv")
            elif self.split=='multi test':
                glove_embedding.to_csv(glove_embedding_save_path[:-4]+"_test_multi.csv")
            elif self.split=='train':
                glove_embedding.to_csv(glove_embedding_save_path[:-4]+"_train_multi.csv")
        
        if self.split=='single test':
             embedding = pd.read_csv('glove-embeddings/preprocessed_glove_embeddings_test_single.csv', encoding='utf-8')
        elif self.split=='multi test':
            embedding = pd.read_csv('glove-embeddings/preprocessed_glove_embeddings_test_multi.csv', encoding='utf-8')
        elif self.split=='train':
            embedding = pd.read_csv('glove-embeddings/preprocessed_glove_embeddings_train_multi.csv', encoding='utf-8')
        
        embedding = embedding.loc[:, ~embedding.columns.str.contains('^Unnamed')]

        return embedding


In [ ]:
dataset={
    'single test':Dataset(data=test_data_single, split='single test'),
    'multi test':Dataset(data=test_data_multi, split='multi test'),
    'train':Dataset(data=train_data, split='train')
}

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1],

**Creating dataloaders using Datasets**

In [ ]:
def get_dataloaders(dataset, single_size):
    dataloaders={}
    
    # Breaking train dataset into train and validation equally from single and muilti word dataset with split = 0.2
    train=dataset['train']
    tokenised_sent_id, sent_attention_mask, complexity, token_embedding, handcrafted_features = sklearn.utils.shuffle(train.tokenised_sent_id[:single_size], train.sent_attention_mask[:single_size], train.complexity[:single_size], train.token_embedding[:single_size], train.handcrafted_features[:single_size], random_state=random_state)
    train_ids, val_ids, _, _ = train_test_split(range(tokenised_sent_id.shape[0]), tokenised_sent_id, test_size = 0.2, random_state = random_state)
    train_data_single = TensorDataset(tokenised_sent_id[train_ids], sent_attention_mask[train_ids], complexity[train_ids], token_embedding[train_ids], handcrafted_features[train_ids])
    val_data_single = TensorDataset(tokenised_sent_id[val_ids], sent_attention_mask[val_ids], complexity[val_ids], token_embedding[val_ids], handcrafted_features[val_ids])
    
    tokenised_sent_id, sent_attention_mask, complexity, token_embedding, handcrafted_features = sklearn.utils.shuffle(train.tokenised_sent_id[single_size:], train.sent_attention_mask[single_size:], train.complexity[single_size:], train.token_embedding[single_size:], train.handcrafted_features[single_size:], random_state=random_state)
    train_ids, val_ids, _, _ = train_test_split(range(tokenised_sent_id.shape[0]), tokenised_sent_id, test_size = 0.2, random_state = random_state)
    train_data_multi = TensorDataset(tokenised_sent_id[train_ids], sent_attention_mask[train_ids], complexity[train_ids], token_embedding[train_ids], handcrafted_features[train_ids])
    val_data_multi = TensorDataset(tokenised_sent_id[val_ids], sent_attention_mask[val_ids], complexity[val_ids], token_embedding[val_ids], handcrafted_features[val_ids])
    
    train_data = ConcatDataset([train_data_single, train_data_multi])
    val_data = ConcatDataset([val_data_single, val_data_multi])

    # Train Dataloader
    train_sampler = RandomSampler(train_data)
    dataloaders['train'] = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Validation Dataloader
    val_sampler = SequentialSampler(val_data)
    dataloaders['val'] = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    # Single word token Test Dataloader
    test=dataset['single test']
    test_data = TensorDataset(test.tokenised_sent_id, test.sent_attention_mask, test.complexity, test.token_embedding, test.handcrafted_features)
    test_sampler = SequentialSampler(test_data) 
    dataloaders['single test'] = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    # Multi word token Test Dataloader
    test=dataset['multi test']
    test_data = TensorDataset(test.tokenised_sent_id, test.sent_attention_mask, test.complexity, test.token_embedding, test.handcrafted_features)
    test_sampler = SequentialSampler(test_data)
    dataloaders['multi test'] = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    return dataloaders


In [ ]:
dataloader = get_dataloaders(dataset, len(train_data_single.id))

Model
> LSTM -> For Glove word embeddings

> BERT -> For Sentence embeddings

> Fully connected layed to combine handcrafted features and generate probability







In [ ]:
class Model(torch.nn.Module):
    def __init__(self, input_dim, n_hidden=100, n_layers=2, dropout=0.25, lr=0.001):
        super(Model, self).__init__()

        self.dropout = dropout
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # LSTM 
        self.lstm =  nn.LSTM(emb_dim, n_hidden, n_layers, dropout=dropout, batch_first=True, bidirectional = True)
        self.linear1 = nn.Linear(self.n_hidden * 2, 500, bias = True)

        # BERT for Sentence
        self.sent_embd = BertModel.from_pretrained('bert-base-uncased',  output_hidden_states = True)
        self.l1 = nn.Linear(input_dim, 500, bias = True)
        self.dropout = nn.Dropout(0.25)
        self.relu = nn.ReLU()

        # Word + Sentence + Handcrafted
        self.linear = nn.Sequential(
                      nn.Linear(1005, 400),
                      nn.Dropout(0.25),
                      nn.ReLU(),
                      nn.Linear(400, 100),
                      nn.Dropout(0.25),
                      nn.ReLU(),
                      nn.Linear(100, 1),
        )
        self.sigmoid = nn.Sigmoid()


    
    def init_hidden(self, batch_size):

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
        hidden = ((torch.zeros(2 * self.n_layers, batch_size, self.n_hidden)).to(device),
                  (torch.zeros(2 * self.n_layers, batch_size, self.n_hidden)).to(device))

        return hidden

    def forward(self, tokenised_sent_id, sent_attention_mask, token_embedding, handcrafted_features, batch_size):
        # LSTM
        hidden = self.init_hidden(batch_size)
        lstm_out, hidden = self.lstm(token_embedding.unsqueeze(dim=1), hidden)
        lstm_out = self.dropout(lstm_out)
        lstm_out = self.linear1(lstm_out)
        y_pred_word = self.relu(lstm_out)
        y_pred_word = y_pred_word.squeeze(dim=1)

        # BERT
        sent_embed = self.sent_embd(tokenised_sent_id,sent_attention_mask)[1]
        sent_embed = self.dropout(sent_embed)
        sent_embed = self.l1(sent_embed)
        y_pred = self.relu(sent_embed)
        y_pred = y_pred.view(y_pred.shape[0], -1)

        # Combine
        comb = torch.cat([y_pred, y_pred_word, handcrafted_features], dim = 1)
        final = self.linear(comb)
        final = final.view(final.shape[0])
        out = self.sigmoid(final)

        return out

**Functions to save and load model metrics**

In [ ]:
def save_metrics(model_save_dir, epochs, model, optimizer, L1):

    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'epochs': epochs+1,
                  'L1': L1}
    
    file_name = model_save_dir+"saved_model_epoch_"+str(epochs)+".pt"
    torch.save(state_dict, file_name)
    print(f'Model saved to ==> {file_name}')

def load_metrics(load_path, model, optimizer):
    try: 
        state_dict = torch.load(load_path, map_location=device)
        model.load_state_dict(state_dict['model_state_dict'])
        optimizer.load_state_dict(state_dict['optimizer_state_dict'])
        print(f'Model loaded from <== {load_path}')
    except: 
        if load_path!='':
            print('pretrained model load failed!!! \n Continiing training without loading ...')
        state_dict = {}

    
    return state_dict.get('epochs', 0), state_dict.get('L1', 1000)

In [ ]:
model = Model(768).to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Train Model**

In [ ]:
# Function to evaluate model on validation dataset
def evaluate(test_dataloader, model):
    model.eval()
    total_eval_accuracy=0

    total_loss = 0
    criterion = nn.L1Loss()

    y_preds = np.array([])
    y_test = np.array([])

    for batch in test_dataloader:
        tokenised_sent_id = batch[0].to(device)
        sent_attention_mask = batch[1].to(device)
        complexity_label = batch[2].to(device)
        token_embedding = batch[3].to(device)
        handcrafted_features = batch[4].to(device)

        with torch.no_grad():
            ypred = model(tokenised_sent_id, sent_attention_mask, token_embedding, handcrafted_features, batch_size = batch[0].to(device).shape[0])        

        ypred = ypred.to('cpu').numpy()
        complexity_label = complexity_label.to('cpu').numpy()

        y_preds = np.hstack((y_preds, ypred))
        y_test = np.hstack((y_test, complexity_label))

    corr, _ = pearsonr(y_preds, y_test)
    loss = np.mean(np.abs(y_preds-y_test))
    return loss, y_preds, y_test, corr
 
def train(training_dataloader, validation_dataloader, model, model_save_dir, load_pretrained_model_path='', epochs = 4):
    optimizer = AdamW(model.parameters(), lr=2e-5, eps = 1e-8)
    criterion = nn.L1Loss()

    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(training_dataloader) * epochs)
    
    best_model = copy.deepcopy(model)

    cur_epoch, best_l1 = load_metrics(load_pretrained_model_path, model, optimizer)                 # Used to continue training from a checkpoint (load_pretrained_model_path must be provided)
    best_epoch = -1

    for epoch_i in range(0, epochs):
        print("Epoch: ",epoch_i,"/",epochs)
        total_train_loss = 0
        model.train()

        for step, batch in tqdm(enumerate(training_dataloader), total=230):
            tokenised_sent_id = batch[0].to(device)
            sent_attention_mask = batch[1].to(device)
            complexity_label = batch[2].to(device)
            token_embedding = batch[3].to(device)
            handcrafted_features = batch[4].to(device)

            outputs = model(tokenised_sent_id, sent_attention_mask, token_embedding, handcrafted_features, batch_size = batch[0].to(device).shape[0])

            loss = criterion(outputs, complexity_label)
 
            if step%50 == 0:
                print("Current Loss - ", loss.item())
 
            total_train_loss += loss
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()


        print(f'Total Train Loss = {total_train_loss}')
        print('#############  Validation  ###########')
        l1_loss, _, _, corr = evaluate(validation_dataloader, model)
        print("  L1 loss: ",l1_loss)
        print("  Pearson correlation: ",corr)
        
        if l1_loss < best_l1:
            best_l1 = l1_loss
            best_epoch = epoch_i
        save_metrics(model_save_dir, epoch_i, model, optimizer, l1_loss)
        print()

    return best_epoch

In [ ]:
best_epoch = train(dataloader['train'], dataloader['val'], model, model_save_dir, epochs=epochs)

Epoch:  0 / 5


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/230 [00:00<?, ?it/s]

Current Loss -  0.17888692021369934
Current Loss -  0.0993976965546608
Current Loss -  0.0858331173658371
Current Loss -  0.09685875475406647
Current Loss -  0.07868648320436478
Total Train Loss = 24.353147506713867
#############  Validation  ###########
  L1 loss:  0.07636124372409295
  Pearson correlation:  0.7462731829943546
Model saved to ==> /content/saves/saved_model_epoch_0.pt

Epoch:  1 / 5


  0%|          | 0/230 [00:00<?, ?it/s]

...

Current Loss -  0.05007670819759369
Current Loss -  0.05150733143091202
Current Loss -  0.0673324465751648
Current Loss -  0.07477529346942902
Current Loss -  0.054621271789073944
Total Train Loss = 14.675580978393555
#############  Validation  ###########
  L1 loss:  0.06863288288336769
  Pearson correlation:  0.810310954695739
Model saved to ==> /content/saves/saved_model_epoch_2.pt

Epoch:  3 / 5


  0%|          | 0/230 [00:00<?, ?it/s]

Current Loss -  0.053717534989118576
Current Loss -  0.055784303694963455
Current Loss -  0.0482986718416214
Current Loss -  0.06610924005508423
Current Loss -  0.05512353032827377
Total Train Loss = 13.524368286132812
#############  Validation  ###########
  L1 loss:  0.06622423095933822
  Pearson correlation:  0.813542263382222
Model saved to ==> /content/saves/saved_model_epoch_3.pt

Epoch:  4 / 5


  0%|          | 0/230 [00:00<?, ?it/s]

Current Loss -  0.05814088135957718
Current Loss -  0.05463613569736481
Current Loss -  0.06458248198032379
Current Loss -  0.051450908184051514
Current Loss -  0.0655025839805603
Total Train Loss = 12.804497718811035
#############  Validation  ###########
  L1 loss:  0.06510389001840279
  Pearson correlation:  0.81844110438831
Model saved to ==> /content/saves/saved_model_epoch_4.pt



In [ ]:
print("Best Epoch -> ",best_epoch)

Best Epoch ->  4


# **TEST**

**Function to generate predictions on test dataset**

In [ ]:
def get_preds(test_dataloader, model):
    model.eval()
    y_preds = []
    total_loss = 0

    for batch in test_dataloader:
        tokenised_sent_id = batch[0].to(device)
        sent_attention_mask = batch[1].to(device)
        complexity_label = batch[2].to(device)
        token_embedding = batch[3].to(device)
        handcrafted_features = batch[4].to(device)

        with torch.no_grad():
            ypred = model(tokenised_sent_id, sent_attention_mask, token_embedding, handcrafted_features, batch_size = batch[0].to(device).shape[0])      
        
        ypred = ypred.to('cpu').numpy()
        y_preds = y_preds + list(ypred)

    return np.array(y_preds)

In [ ]:
model = Model(768).to(device)

###**Function to generate scores on predicted labels** 
(reference -> https://github.com/MMU-TDMLab/CompLex/blob/master/evaluate.py)

Note: 

*   Generated scores will be saved inside score directory (output format -> ```score/scores_<split_type>_epoch_<epoch_no>.txt```)
*   Predicted labels will be saved as ```predictions_<split_type>_epoch_<epoch_no>.csv'``` in the main directory (i.e. content dir of colab)




In [ ]:
def generate_scores(split, filename, epoch_no):
    submission_dict = {}
    with open(filename, 'r') as sf:
        csv_reader = csv.reader(sf)
        for item in csv_reader:
            try:
                submission_dict[item[0]] = float(item[1])
            except:
                # print(item)
                pass

    #Read reference:
    gold_dict = {}
    if split=='single test':
        path='test-labels/lcp_single_test.tsv'
    else:
        path='test-labels/lcp_multi_test.tsv'

    reference_file = pd.read_csv(path,  delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
    for id, complexity in zip(list(reference_file.id), list(reference_file.complexity)):
        gold_dict[id] = float(complexity)

    #Produce vectors with reference labels and predictions:
    gold = []
    predicted = []
    
    for key, label in gold_dict.items():
        gold.append(label)
        predicted.append(submission_dict[key])
    
    #Calculate scores:
    pearson_score = pearsonr(gold, predicted)[0]
    spearman_score = spearmanr(gold, predicted)[0]
    mae_score = mean_absolute_error(gold, predicted)
    mse_score = mean_squared_error(gold, predicted)
    rsq_score = r2_score(gold, predicted)

    result = ''
    result += "pearson:{0}\n".format(pearson_score)
    result += "spearman:{0}\n".format(spearman_score)
    result += "mae:{0}\n".format(mae_score)
    result += "mse:{0}\n".format(mse_score)
    result += "r2:{0}".format(rsq_score)

    print(result)
    with open('score/scores_'+split+'_epoch_'+str(epoch_no)+'.txt', 'w') as output_file:
        output_file.write(result)

**Function to generate test results and save predicted labels**

In [ ]:
def generate_test_result(model, epoch_no, split):
    print('\n#### Result for {split} at epoch {epoch}'.format(split=split, epoch=epoch_no))

    state_dict = torch.load(f = '/content/saves/saved_model_epoch_'+str(epoch_no)+'.pt', map_location=device)
    model.load_state_dict(state_dict['model_state_dict'])

    preds = get_preds(dataloader[split], model)

    results= pd.DataFrame()
    results['id'] = list(dataset[split].data.id)
    results['complexity'] = list(preds)
    save_file = 'predictions_'+split+'_epoch_'+str(epoch_no)+'.csv'
    results.to_csv(save_file, header=False, index=False)

    generate_scores(split, save_file, epoch_no)

In [ ]:
generate_test_result(model, epoch_no=best_epoch, split='single test')
generate_test_result(model, epoch_no=best_epoch, split='multi test')


#### Result for single test at epoch 4
pearson:0.744538341157708
spearman:0.7051862023408643
mae:0.06636416507132929
mse:0.007319918959110097
r2:0.5477364939428904

#### Result for multi test at epoch 4
pearson:0.8031755769805867
spearman:0.8005849017999792
mae:0.0730116769895406
mse:0.008621047001439397
r2:0.6428305171820734


References:
Original code base: Team 13
https://colab.research.google.com/drive/1rRv2EZ34_KXWCGrKvRKI0VEDYQ1WL_Sd?usp=sharing#scrollTo=7UZm4aGBberR

Sense extraction:

credited to
Author: Abdelrahman Khaled Hussen PDH
Date: 4/2/2017

Wordnet features
https://www.nltk.org/howto/wordnet.html
